In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

In [2]:
# Directories where your data are stored
train_data_dir = r'C:\Users\ABDERRAHMANE\Desktop\fiverr-work\New Plant Diseases Dataset(Augmented)\train'
valid_data_dir = r'C:\Users\ABDERRAHMANE\Desktop\fiverr-work\New Plant Diseases Dataset(Augmented)\valid'

def load_images_from_folder(folder):
    images = []
    labels = []
    for folder1 in os.listdir(folder):
        label = folder1
        folder1_path = os.path.join(folder, folder1)
        for img_name in os.listdir(folder1_path):
            img = cv2.imread(os.path.join(folder1_path, img_name))
            img = cv2.resize(img, (64, 64))
            images.append(img)
            labels.append(label)
    images = np.array(images) / 255.0
    encoder = LabelEncoder()
    labels = encoder.fit_transform(labels)
    labels = to_categorical(labels)
    return images, labels, encoder.classes_

# Load training and validation data
x_train, y_train, class_names_train = load_images_from_folder(train_data_dir)
x_valid, y_valid, class_names_valid = load_images_from_folder(valid_data_dir)

class_names = class_names_train if len(class_names_train) > len(class_names_valid) else class_names_valid

# Define the CNN model
def create_model():
    model = Sequential([
        Conv2D(32, 3, activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D(),
        Conv2D(64, 3, activation='relu'),
        MaxPooling2D(),
        Conv2D(128, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')  # Number of classes in the dataset
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Use Stratified KFold for training
skf = StratifiedKFold(n_splits=2, shuffle=True)
for train_index, val_index in skf.split(x_train, np.argmax(y_train, axis=1)):
    partial_x_train, x_val = x_train[train_index], x_train[val_index]
    partial_y_train, y_val = y_train[train_index], y_train[val_index]
    model = create_model()
    model.fit(partial_x_train, partial_y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(x_valid, y_valid)
print(f'Validation accuracy: {accuracy}')

# Save the trained model
model_path = r'C:\Users\ABDERRAHMANE\Desktop\fiverr-work\New Plant Diseases Dataset(Augmented)\model_save.h5'
model.save(model_path)

Epoch 1/10
122/122 [==============================] - 28s 207ms/step - loss: 0.8637 - accuracy: 0.6309 - val_loss: 0.5422 - val_accuracy: 0.7867
Epoch 2/10
122/122 [==============================] - 23s 191ms/step - loss: 0.4672 - accuracy: 0.8242 - val_loss: 0.3690 - val_accuracy: 0.8636
Epoch 3/10
122/122 [==============================] - 23s 192ms/step - loss: 0.3082 - accuracy: 0.8875 - val_loss: 0.2807 - val_accuracy: 0.8948
Epoch 4/10
122/122 [==============================] - 25s 203ms/step - loss: 0.2382 - accuracy: 0.9156 - val_loss: 0.2172 - val_accuracy: 0.9223
Epoch 5/10
122/122 [==============================] - 23s 193ms/step - loss: 0.1648 - accuracy: 0.9372 - val_loss: 0.1304 - val_accuracy: 0.9560
Epoch 6/10
122/122 [==============================] - 24s 198ms/step - loss: 0.1583 - accuracy: 0.9385 - val_loss: 0.1739 - val_accuracy: 0.9367
Epoch 7/10
122/122 [==============================] - 25s 205ms/step - loss: 0.1270 - accuracy: 0.9519 - val_loss: 0.1660 - val_ac

In [ ]:
# Start video capture
video_capture = cv2.VideoCapture(0)

try:
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()

        # Preprocess the image (you need to insert your preprocessing here)
        frame_resized = cv2.resize(frame, (64, 64))  # for instance if you resized your images to 64x64
        frame_normalized = frame_resized / 255.0  # and if you normalized pixel values
        
        # Remember that the model expects a batch of images as input,
        # so we need to add an extra dimension
        img = np.expand_dims(frame_normalized, axis=0)

        # Make a prediction
        prediction = model.predict(img)

        # Determine the class with the highest probability
        predicted_class = np.argmax(prediction, axis=1)

        # Display the prediction on the frame
        cv2.putText(frame, f'Prediction: {class_names[predicted_class[0]]}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Convert the image to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale image (you can adjust the kernel size)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Use adaptive thresholding (you can adjust the block size and C value)
        thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

        # Find contours in the thresholded image
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Draw contours on the original image
        cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # When everything is done, release the capture and destroy the windows
    video_capture.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 37ms/step
